# Fetch Historical Data
Fetches historical data from the Trafiklab API Kollektivtrafikens Datalabb (KoDa).

## Imports

In [1]:
import os
from pathlib import Path
import sys

root_dir = Path().absolute()
# Strip subdirectories if the notebook started in any
if root_dir.parts[-1:] == ('pipeline',):
    root_dir = Path(*root_dir.parts[:-1])
if root_dir.parts[-1:] == ('src',):
    root_dir = Path(*root_dir.parts[:-1])
root_dir = str(root_dir) 

os.chdir(root_dir)
print(f"Root dir: {Path.cwd()}")

from datetime import date, timedelta
from src.data_utils.filter import *
from src.data_utils.ingest import *

Root dir: C:\Users\royli\Desktop\Courses\ID2223_Scalable_Machine_Learning_and_Deep_Learning\Project


## Fetch Data

### Decide How Many Days in the Past to Fetch

In [2]:
# Define the start date and how many days in the past including the start date to fetch
# Start date must be yesterday or earlier
number_of_days = 7
start_date = date(2025, 12, 25)

dates = [start_date - timedelta(days=i) for i in range(number_of_days)]
dates = [d.strftime("%Y-%m-%d") for d in dates]

print(dates)

['2025-12-25', '2025-12-24', '2025-12-23', '2025-12-22', '2025-12-21', '2025-12-20', '2025-12-19']


### Fetch Static Data from Trafiklab's KoDa API

In [3]:
DATA_ROOT = Path('data')
for d in dates:
    date_dir = Path(f"data/static/{d}")
    if date_dir.exists():
        print(f"{d} exists, skipping")
        continue

    print(f"Fetching static data from {d}")
    max_retries = 10
    for attempt in range(max_retries):
        try:
            zip_file = fetch_static(d, "data")
            zip_dir = extract_zip(zip_file)
            output_dir = txt_to_csv(zip_dir)
            break
        except Exception as e:
            print(f"Attempt {attempt} failed: {e}")
            if attempt == max_retries - 1:
                print("Reached max retries, skipping date")
                
            sleep_time = 5
            print(f"Retrying in {sleep_time}s...")
            time.sleep(sleep_time)
    else:
        print(f'Download failed for date {d}')
        break
    filter_irrelevant_files(output_dir, date)
    filter_static_data_for_date(output_dir, date)
print("Finished")

2025-12-25 exists, skipping
2025-12-24 exists, skipping
2025-12-23 exists, skipping
2025-12-22 exists, skipping
2025-12-21 exists, skipping
2025-12-20 exists, skipping
2025-12-19 exists, skipping
Finished


### Fetch Realtime VehiclePositions Data from Trafiklab's KoDa API

In [4]:
for d in dates:
    date_dir = Path(f"data/realtime/{d}/VehiclePositions")
    if date_dir.exists():
        print(f"{d} exists, skipping")
        continue

    print(f"Fetching realtime VehiclePositions data for {d}")
    realtime_file = fetch_realtime(d, "data", feed='VehiclePositions')
    if realtime_file is not None:
        extracted = extract_7z(realtime_file, feed='VehiclePositions')
        raw_dir = flatten_extracted_structure(extracted)
        filter_realtime_data_VP(raw_dir, d)

print("Finished fetching realtime VehiclePositions data")

Fetching realtime VehiclePositions data for 2025-12-25
[1/65] Data is being prepared, retrying connection in 60 seconds
[2/65] Still processing, retrying connection in 60 seconds
[3/65] Still processing, retrying connection in 60 seconds
[4/65] Still processing, retrying connection in 60 seconds
[5/65] Still processing, retrying connection in 60 seconds
[6/65] Still processing, retrying connection in 60 seconds
[7/65] Still processing, retrying connection in 60 seconds
[8/65] Still processing, retrying connection in 60 seconds


KeyboardInterrupt: 

In [4]:
for d in dates:
    date_dir = Path(f"data/realtime/{d}/TripUpdates")
    if date_dir.exists():
        print(f"{d} exists, skipping")
        continue

    print(f"Fetching realtime TripUpdates data for {d}")
    realtime_file = fetch_realtime(d, "data", feed='TripUpdates')
    if realtime_file is not None:
        extracted = extract_7z(realtime_file, feed='TripUpdates')
        raw_dir = flatten_extracted_structure(extracted)
        filter_realtime_data_TU(raw_dir, d)

print("Finished fetching realtime TripUpdates data")

Fetching realtime TripUpdates data for 2025-12-25
[1/65] Data is being prepared, retrying connection in 60 seconds
[2/65] Data is being prepared, retrying connection in 60 seconds
[3/65] Data is being prepared, retrying connection in 60 seconds
[4/65] Data is being prepared, retrying connection in 60 seconds
[5/65] Data is being prepared, retrying connection in 60 seconds
[6/65] Data is being prepared, retrying connection in 60 seconds
[7/65] Data is being prepared, retrying connection in 60 seconds
[8/65] Data is being prepared, retrying connection in 60 seconds
[9/65] Data is being prepared, retrying connection in 60 seconds
[10/65] Data is being prepared, retrying connection in 60 seconds
[11/65] Data is being prepared, retrying connection in 60 seconds
[12/65] Data is being prepared, retrying connection in 60 seconds
[13/65] Data is being prepared, retrying connection in 60 seconds
[14/65] Data is being prepared, retrying connection in 60 seconds
[15/65] Data is being prepared, ret

Filtering snapshots for hour 00: 100%|███████████████████████████████████████████████| 255/255 [00:41<00:00,  6.17it/s]


Done filtering RT VP json files
Writing snapshots to 00.json
Preprocessed, filtered and aggregated hour 00 with 0 snapshots
removing directory data\realtime\2025-12-21\TripUpdates\raw\00
Deleted raw directory data\realtime\2025-12-21\TripUpdates\raw\00
Converted data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-13Z.pb to data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-13Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-27Z.pb to data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-27Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-41Z.pb to data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-41Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-55Z.pb to data\realtime\2025-12-21\TripUpdates\raw\01\sl-tripupdates-2025-12-21T01-00-55Z.json.
Converted data\real

Filtering snapshots for hour 01: 100%|███████████████████████████████████████████████| 252/252 [00:33<00:00,  7.62it/s]


Done filtering RT VP json files
Writing snapshots to 01.json
Preprocessed, filtered and aggregated hour 01 with 0 snapshots
removing directory data\realtime\2025-12-21\TripUpdates\raw\01
Deleted raw directory data\realtime\2025-12-21\TripUpdates\raw\01
Converted data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-12Z.pb to data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-12Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-26Z.pb to data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-26Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-40Z.pb to data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-00-40Z.json.
Converted data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-01-08Z.pb to data\realtime\2025-12-21\TripUpdates\raw\02\sl-tripupdates-2025-12-21T02-01-08Z.json.
Converted data\real

...

Done filtering RT VP json files
Writing snapshots to 21.json
Preprocessed, filtered and aggregated hour 21 with 0 snapshots
removing directory data\realtime\2025-12-20\TripUpdates\raw\21
Deleted raw directory data\realtime\2025-12-20\TripUpdates\raw\21
Converted data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-06Z.pb to data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-06Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-19Z.pb to data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-19Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-33Z.pb to data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-33Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-47Z.pb to data\realtime\2025-12-20\TripUpdates\raw\22\sl-tripupdates-2025-12-20T22-00-47Z.json.
Converted data\real

Filtering snapshots for hour 22: 100%|███████████████████████████████████████████████| 254/254 [00:28<00:00,  8.87it/s]


Done filtering RT VP json files
Writing snapshots to 22.json
Preprocessed, filtered and aggregated hour 22 with 0 snapshots
removing directory data\realtime\2025-12-20\TripUpdates\raw\22
Deleted raw directory data\realtime\2025-12-20\TripUpdates\raw\22
Converted data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-04Z.pb to data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-04Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-17Z.pb to data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-17Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-31Z.pb to data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-31Z.json.
Converted data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-45Z.pb to data\realtime\2025-12-20\TripUpdates\raw\23\sl-tripupdates-2025-12-20T23-00-45Z.json.
Converted data\real

Filtering snapshots for hour 23: 100%|███████████████████████████████████████████████| 248/248 [00:25<00:00,  9.88it/s]


Done filtering RT VP json files
Writing snapshots to 23.json
Preprocessed, filtered and aggregated hour 23 with 0 snapshots
removing directory data\realtime\2025-12-20\TripUpdates\raw\23
Deleted raw directory data\realtime\2025-12-20\TripUpdates\raw\23
Fetching realtime TripUpdates data for 2025-12-19
[1/65] Data is being prepared, retrying connection in 60 seconds
[2/65] Data is being prepared, retrying connection in 60 seconds
[3/65] Data is being prepared, retrying connection in 60 seconds
[4/65] Data is being prepared, retrying connection in 60 seconds
[5/65] Data is being prepared, retrying connection in 60 seconds
[6/65] Data is being prepared, retrying connection in 60 seconds
[7/65] Data is being prepared, retrying connection in 60 seconds
[8/65] Data is being prepared, retrying connection in 60 seconds
[9/65] Data is being prepared, retrying connection in 60 seconds
[10/65] Data is being prepared, retrying connection in 60 seconds
[11/65] Data is being prepared, retrying conne